<a href="https://www.kaggle.com/code/aayushsin7a/cirrhosis-patient-survival-prediction?scriptVersionId=154669269" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cirrhosis-prediction-dataset/cirrhosis.csv
/kaggle/input/cirrhosis-patient-survival-data-dictionary/bank-marketing-classification.ipynb
/kaggle/input/cirrhosis-patient-survival-data-dictionary/scania-aps-failure-prediction.ipynb
/kaggle/input/cirrhosis-patient-survival-data-dictionary/nyc-taxi-fare-prediction.ipynb
/kaggle/input/cirrhosis-patient-survival-data-dictionary/Cirrhosis Prediction.xlsx
/kaggle/input/cirrhosis-patient-survival-dd/bank-marketing-classification.ipynb
/kaggle/input/cirrhosis-patient-survival-dd/scania-aps-failure-prediction.ipynb
/kaggle/input/cirrhosis-patient-survival-dd/nyc-taxi-fare-prediction.ipynb
/kaggle/input/cirrhosis-patient-survival-dd/Cirrhosis Prediction.xlsx
/kaggle/input/cirrhosis-patient-survival-data-description/Cirrhosis Prediction.xlsx


> # Import the required libraries


In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np 
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
sns.set_theme(style='whitegrid')
import re
import calendar
# Set display options to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
# Ignore all warnings
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
dataDictionary = pd.read_excel("/kaggle/input/cirrhosis-patient-survival-data-dictionary/Cirrhosis Prediction.xlsx")

In [4]:
dataDictionary

,Variable Name,Role,Type,Demographic,Description,Units,Missing Values
0,ID,ID,Integer,-,Unique identifier,-,No
1,N_Days,Other,Integer,-,Number of days between registration and the ea...,-,No
2,Drug,Feature,Categorical,-,Type of drug: D-penicillamine or placebo. Type...,-,Yes
3,Age,Feature,Integer,Age,Age. Age could be related to disease progressi...,Days,No
4,Sex,Feature,Categorical,Sex,Gender: M (male) or F (female). Biological sex...,-,No
5,Ascites,Feature,Categorical,-,Presence of ascites: N (No) or Y (Yes). The ac...,-,Yes
6,Hepatomegaly,Feature,Categorical,-,Presence of hepatomegaly: N (No) or Y (Yes). E...,-,Yes
7,Spiders,Feature,Categorical,-,Presence of spiders: N (No) or Y (Yes). Spider...,-,Yes
8,Edema,Feature,Categorical,-,Presence of edema: N (no edema and no diuretic...,-,No
9,Bilirubin,Feature,Continuous,-,Serum bilirubin. High levels can indicate live...,mg/dL,No


In [5]:
df = pd.read_csv("/kaggle/input/cirrhosis-prediction-dataset/cirrhosis.csv")

In [6]:
import pandas as pd

def diagnose_dataframe(df, verbose=True):
  """
  Performs various initial inspections on a pandas dataframe and provides details.

  Args:
    df: The pandas dataframe to diagnose.
    verbose: Whether to print details for each inspection.

  Returns:
    A dictionary containing the results of the inspections.
  """

  results = {}

  # Basic information
  results["shape"] = df.shape
  results["columns"] = list(df.columns)
  results["dtypes"] = df.dtypes.to_dict()
  results["null_counts"] = df.isnull().sum().to_dict()

  # Descriptive statistics
  results["describe"] = df.describe(include='all')

  # Unique values and counts
  for col in df.columns:
    results[f"unique_values_{col}"] = df[col].unique()
    results[f"value_counts_{col}"] = df[col].value_counts().to_dict()

  # Missing value analysis
  missing_values_pct = (df.isnull().sum() / df.shape[0]) * 100
  results["missing_values_pct"] = missing_values_pct.to_dict()
  missing_values_threshold = 0.5  # Set a threshold for high missing value percentage
  high_missing_values = missing_values_pct[missing_values_pct > missing_values_threshold]
  results["high_missing_values"] = list(high_missing_values.index)

  # Duplicate rows
  duplicate_rows = df[df.duplicated()]
  results["duplicate_rows_count"] = len(duplicate_rows)

  # Outlier analysis
  # ... (implement your own outlier analysis strategy)

  if verbose:
    for key, value in results.items():
      print(f"\n{key}:\n{value}")

  return results

In [7]:
diagnose_dataframe(df)


shape:
(418, 20)

columns:
['ID', 'N_Days', 'Status', 'Drug', 'Age', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin', 'Stage']

dtypes:
{'ID': dtype('int64'), 'N_Days': dtype('int64'), 'Status': dtype('O'), 'Drug': dtype('O'), 'Age': dtype('int64'), 'Sex': dtype('O'), 'Ascites': dtype('O'), 'Hepatomegaly': dtype('O'), 'Spiders': dtype('O'), 'Edema': dtype('O'), 'Bilirubin': dtype('float64'), 'Cholesterol': dtype('float64'), 'Albumin': dtype('float64'), 'Copper': dtype('float64'), 'Alk_Phos': dtype('float64'), 'SGOT': dtype('float64'), 'Tryglicerides': dtype('float64'), 'Platelets': dtype('float64'), 'Prothrombin': dtype('float64'), 'Stage': dtype('float64')}

null_counts:
{'ID': 0, 'N_Days': 0, 'Status': 0, 'Drug': 106, 'Age': 0, 'Sex': 0, 'Ascites': 106, 'Hepatomegaly': 106, 'Spiders': 106, 'Edema': 0, 'Bilirubin': 0, 'Cholesterol': 134, 'Albumin': 0, 'Copper': 108, 

{'shape': (418, 20),
 'columns': ['ID',
  'N_Days',
  'Status',
  'Drug',
  'Age',
  'Sex',
  'Ascites',
  'Hepatomegaly',
  'Spiders',
  'Edema',
  'Bilirubin',
  'Cholesterol',
  'Albumin',
  'Copper',
  'Alk_Phos',
  'SGOT',
  'Tryglicerides',
  'Platelets',
  'Prothrombin',
  'Stage'],
 'dtypes': {'ID': dtype('int64'),
  'N_Days': dtype('int64'),
  'Status': dtype('O'),
  'Drug': dtype('O'),
  'Age': dtype('int64'),
  'Sex': dtype('O'),
  'Ascites': dtype('O'),
  'Hepatomegaly': dtype('O'),
  'Spiders': dtype('O'),
  'Edema': dtype('O'),
  'Bilirubin': dtype('float64'),
  'Cholesterol': dtype('float64'),
  'Albumin': dtype('float64'),
  'Copper': dtype('float64'),
  'Alk_Phos': dtype('float64'),
  'SGOT': dtype('float64'),
  'Tryglicerides': dtype('float64'),
  'Platelets': dtype('float64'),
  'Prothrombin': dtype('float64'),
  'Stage': dtype('float64')},
 'null_counts': {'ID': 0,
  'N_Days': 0,
  'Status': 0,
  'Drug': 106,
  'Age': 0,
  'Sex': 0,
  'Ascites': 106,
  'Hepatomegaly